In [1]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd

# Import the API key
from config import geoapify_key

In [2]:
zipcode = "90049" #input chosen zipcode here

params = {
    "apiKey": geoapify_key,
    "country":"United States",
    "postcode": zipcode
}
# Build URL using the geocode endpoint
base_url = f"https://api.geoapify.com/v1/geocode/search?text={zipcode}&lang=en&type=postcode&filter=countrycode:us"

In [3]:
#run request
response = requests.get(base_url, params=params).json()

In [4]:
 # Extract lat/lon
lat = response["features"][0]["properties"]["lat"]
lon = response["features"][0]["properties"]["lon"]
place_id = response["features"][0]["properties"]["place_id"]

# Print results
print(f"{zipcode}: {lat}, {lon}")
print(place_id)

90049: 34.07755113, -118.469933765
5183fd1165139e5dc059168f0732ed094140f00101f901bf0c060000000000c00207


In [5]:
# Set paramaters for type of place
categories = "education.school"

# Set parameters for type of search
bias = f"proximity:{lon},{lat}"
filters = f"place:{place_id}"

In [6]:
#parameters dictionary
params = {
    "categories": categories,
    "bias": bias,
    "filter": filters,
    "apiKey": geoapify_key    
}

In [7]:
# Set base URL for places api
base_url = f"https://api.geoapify.com/v2/places?filter=place:{place_id}"

In [8]:
response = requests.get(base_url, params=params).json()

In [9]:
# Extract the features list from the dictionary
features_list = response['features']

# Extract the relevant properties from each feature
rows = []
for feature in features_list:
    properties = feature['properties']
    rows.append(properties)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(rows)

In [10]:
df = df.dropna(subset=['name', 'postcode'])
df = df.drop(columns=['country','country_code','state','street','lon','lat','state_code','formatted','address_line1',
                     'address_line2','categories','details','datasource','place_id','housenumber','district','suburb',
                      'quarter','old_name','city'])

In [11]:
area_df = df.rename(columns={
    'postcode': 'zipcode'
})
area_df

,county,zipcode,distance,name
1,Los Angeles County,90049,855,Bellagio Newcomer Center
2,Los Angeles County,90049,1074,Brentwood School East Campus
4,Los Angeles County,90049,1129,Brentwood School Athletic Facilities VA lease
5,Los Angeles County,90049,1253,The John Thomas Dye School
6,Los Angeles County,90049,1429,Mount Saint Mary's College - Chalon Campus
7,Los Angeles County,90049,1718,Saint Martin of Tours School
8,Los Angeles County,90272,1851,Kenter Canyon Elementary School
9,Los Angeles County,90272,1890,Marymount Junior School
11,Los Angeles County,90077,2273,Marymount High School
12,Los Angeles County,90095,2372,UCLA Lab School


In [12]:
# Create a new data frame that only includes rows with the target zip code
zip_df = area_df.loc[area_df['zipcode'] == zipcode]
zip_df

,county,zipcode,distance,name
1,Los Angeles County,90049,855,Bellagio Newcomer Center
2,Los Angeles County,90049,1074,Brentwood School East Campus
4,Los Angeles County,90049,1129,Brentwood School Athletic Facilities VA lease
5,Los Angeles County,90049,1253,The John Thomas Dye School
6,Los Angeles County,90049,1429,Mount Saint Mary's College - Chalon Campus
7,Los Angeles County,90049,1718,Saint Martin of Tours School
